In [1]:
import pandas as pd
import sys
from datetime import date
from datetime import datetime
from python_calamine import CalamineWorkbook
pd.set_option('display.max_columns', 40)

print(sys.prefix)

C:\Users\josue\OneDrive\Documents\eDNA Challenges\HotelRevenueManagement\.venv


### Load Excel files

In [2]:
workbook_path = "../data/Hotel Revenue Management raw.xlsx"

In [3]:
# Verify original dataset
workbook = CalamineWorkbook.from_path(workbook_path)
print("Path " + workbook.path)
print("Found sheets " + str(workbook.sheet_names))
print("Sheet metadata \n" + str(workbook.sheets_metadata))

Path ../data/Hotel Revenue Management raw.xlsx
Found sheets ['2018', '2019', '2020', 'market_segment', 'meal_cost']
Sheet metadata 
[SheetMetadata(name='2018', typ=SheetTypeEnum.WorkSheet, visible=SheetVisibleEnum.Visible), SheetMetadata(name='2019', typ=SheetTypeEnum.WorkSheet, visible=SheetVisibleEnum.Visible), SheetMetadata(name='2020', typ=SheetTypeEnum.WorkSheet, visible=SheetVisibleEnum.Visible), SheetMetadata(name='market_segment', typ=SheetTypeEnum.WorkSheet, visible=SheetVisibleEnum.Visible), SheetMetadata(name='meal_cost', typ=SheetTypeEnum.WorkSheet, visible=SheetVisibleEnum.Visible)]


In [4]:
# Load sheets into individual data frames
with pd.ExcelFile(workbook_path, engine='calamine') as file:
    data2018 = pd.read_excel(file, sheet_name='2018')
    data2019 = pd.read_excel(file, sheet_name='2019')
    data2020 = pd.read_excel(file, sheet_name='2020')
    market_data = pd.read_excel(file, sheet_name='market_segment')
    meal_cost_dta = pd.read_excel(file, sheet_name='meal_cost')
    file.close()

### Verify structure and Unify datasets

In [5]:
data2018.shape , data2019.shape, data2020.shape

((21815, 32), (79083, 32), (40687, 32))

In [6]:
#Verify column names
data2018.columns == data2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True])

In [7]:
print(f"Column in 2018: {data2018.columns[27]}" )
print(f"Column in 2019: {data2019.columns[27]}" )

Column in 2018: AVG Daily Rate
Column in 2019: adr


In [8]:
column_names = data2018.columns.to_list()
column_names

['hotel',
 'is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'agent',
 'company',
 'days_in_waiting_list',
 'customer_type',
 'AVG Daily Rate',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status',
 'reservation_status_date']

In [9]:
# use standard names for all dataframes
data2018.rename(columns={'AVG Daily Rate' : 'avg_daily_rate'}, inplace=True)
data2019.rename(columns={'adr': 'avg_daily_rate'}, inplace=True)
data2020.rename(columns={'adr' : 'avg_daily_rate'}, inplace=True)

In [10]:
# Sum of all datasets is a total of rows
data2018.shape[0] + data2019.shape[0] + data2020.shape[0]

141585

In [11]:
# Union all datasets into a single master one, removing duplicates
masterdataset = pd.concat([data2018, data2019, data2020]).drop_duplicates()

In [12]:
masterdataset.shape

(100732, 32)

In [13]:
# Convert month names into numbers
masterdataset['arrival_date_month_number'] = masterdataset.apply(lambda row: datetime.strptime(row['arrival_date_month'], '%B').month, axis=1)

In [14]:
#generate arrival_date from dseparated ate pieces 
masterdataset['arrival_date'] = masterdataset.apply(lambda row: datetime(row['arrival_date_year'], row['arrival_date_month_number'], row['arrival_date_day_of_month']), axis=1)

In [15]:
# Finally clean up dataset with date columns no longer useful
masterdataset.drop(columns=['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month', 'arrival_date_month_number'], inplace=True)

### Perform a quick Completeness test

In [16]:
# Perform a quick completeness test

# We can observe that most columns match with total of rows expected (100,732), except:
# Children (100724)
# Country (100171)
# Agent (86193)
# Company (6161)

masterdataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100732 entries, 0 to 40686
Data columns (total 30 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           100732 non-null  object        
 1   is_canceled                     100732 non-null  int64         
 2   lead_time                       100732 non-null  int64         
 3   arrival_date_week_number        100732 non-null  int64         
 4   stays_in_weekend_nights         100732 non-null  int64         
 5   stays_in_week_nights            100732 non-null  int64         
 6   adults                          100732 non-null  int64         
 7   children                        100724 non-null  float64       
 8   babies                          100732 non-null  int64         
 9   meal                            100732 non-null  object        
 10  country                         100171 non-null  object       

In [17]:
# We could ask why those values are missing

# So our assumptions for now:
# Missing Country info is not sensitive, this could be because guests simply didn't fill that info during reservation
# Missing  Company makes sense for non-related work accomodations. 
# Missing Agent also makes sense since many reservations get through direct channel rather than an intermediary Booking Company
# Missing children isn't harmful, so for those NAs, we assume they are zero. This way we can perform math operations over full column
masterdataset.fillna(value= {'children' : 0}, inplace=True)

In [18]:
from_date = masterdataset.reservation_status_date.min()
to_date = masterdataset.reservation_status_date.max()
from_date, to_date

(Timestamp('2018-01-01 00:00:00'), Timestamp('2020-09-14 00:00:00'))

In [19]:
days_between = (to_date - from_date).days
days_between

987

In [20]:
unique_dates_dataset = len(masterdataset.reservation_status_date.unique())
unique_dates_dataset

923

In [21]:
# Be aware, we have 64 missing days with reservations in our dataset. 
# It's hard to beleive that there were no reservations at all in such days.
# Keep it in mind when doing exploratory analysis
days_between - unique_dates_dataset

64

In [22]:
date_range = pd.date_range(start=from_date, end=to_date, freq='D')
masterdataset_dates = pd.DatetimeIndex(data=masterdataset.reservation_status_date.unique())

In [23]:
# Missing days outcome by difference between 2 sets

date_range_set = set(date_range)
masterdataset_dates_set = set(masterdataset_dates)
missing_days = list(sorted(date_range_set - masterdataset_dates_set))
missing_days

[Timestamp('2018-01-03 00:00:00'),
 Timestamp('2018-01-04 00:00:00'),
 Timestamp('2018-01-05 00:00:00'),
 Timestamp('2018-01-06 00:00:00'),
 Timestamp('2018-01-07 00:00:00'),
 Timestamp('2018-01-08 00:00:00'),
 Timestamp('2018-01-09 00:00:00'),
 Timestamp('2018-01-10 00:00:00'),
 Timestamp('2018-01-11 00:00:00'),
 Timestamp('2018-01-12 00:00:00'),
 Timestamp('2018-01-13 00:00:00'),
 Timestamp('2018-01-14 00:00:00'),
 Timestamp('2018-01-15 00:00:00'),
 Timestamp('2018-01-16 00:00:00'),
 Timestamp('2018-01-17 00:00:00'),
 Timestamp('2018-01-19 00:00:00'),
 Timestamp('2018-01-23 00:00:00'),
 Timestamp('2018-01-24 00:00:00'),
 Timestamp('2018-01-25 00:00:00'),
 Timestamp('2018-01-26 00:00:00'),
 Timestamp('2018-01-27 00:00:00'),
 Timestamp('2018-01-31 00:00:00'),
 Timestamp('2018-02-01 00:00:00'),
 Timestamp('2018-02-03 00:00:00'),
 Timestamp('2018-02-04 00:00:00'),
 Timestamp('2018-02-07 00:00:00'),
 Timestamp('2018-02-08 00:00:00'),
 Timestamp('2018-02-13 00:00:00'),
 Timestamp('2018-02-

### Export clean dataset

In [24]:
masterdataset.to_csv(path_or_buf="../data/HotelMasterDataset.csv", index_label="reservation_id")